In [16]:
# Импортируем библиотеки
import os
from datetime import datetime
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_openai import ChatOpenAI

# Получаем ключ API
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("OPENAI_API_KEY not found in environment variables.")

# Функция получения текущего времени
@tool
def get_current_time() -> dict:
    """
    Return the current UTC time in ISO‑8601 format.
    Example → {"utc": "2025‑05‑21T06:42:00Z"}
    """
    return {"utc": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")}

tools = [get_current_time]

# Настройка модели и связывание с инструментами
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    openai_api_key=OPENAI_API_KEY
).bind_tools(tools)

tool_node = ToolNode(tools)

# Функция вызова мдели
def chat(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

# Функция проверки на необходимость вызова инструмента
def should_continue(state: MessagesState):
    last = state["messages"][-1]
    if hasattr(last, "tool_calls") and last.tool_calls:
        return "tools"
    return END

# Построение графа
graph = StateGraph(MessagesState)
graph.add_node("chat", chat)
graph.add_node("tools", tool_node)
graph.add_edge(START, "chat")
graph.add_conditional_edges("chat", should_continue, ["tools", END])
graph = graph.compile()

# Основной цикл
if __name__ == "__main__":
    system = SystemMessage(content="You are a helpful assistant. Use the get_current_time tool when asked about time")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye :)")
            break
        user_msg = HumanMessage(content=user_input)
        result = graph.invoke({"messages": [system, user_msg]})
        for msg in result["messages"]:
            if isinstance(msg, AIMessage) and msg.content:
                print("Bot:", msg.content)


You: What time is it?
You: What time is it?


KeyboardInterrupt: Interrupted by user